In [62]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import cv2 as cv
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import Image

In [21]:
classifier = Sequential()
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3),
                      activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3),
                      activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [27]:
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate = 0.2))
classifier.add(Dense(units = 2, activation = 'softmax'))

In [28]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [29]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.3,
    zoom_range = 0.3,
    horizontal_flip = True,
    validation_split = 0.2
)

# test_datagen = ImageDataGenerator(rescale = 1. / 255)

In [89]:
training_set = train_datagen.flow_from_directory('',
                                                 target_size = (96, 96),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset = 'training')
test_set = train_datagen.flow_from_directory('tr',
                                                 target_size = (96, 96),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [82]:
classifier.fit_generator(training_set,
               steps_per_epoch=100,
               epochs=1,
               validation_data=test_set, 
               validation_steps=24)

/Users/sanzhar/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


100/100 [==============================] - 25s 246ms/step - loss: 2.5576e-05 - accuracy: 1.0000 - val_loss: 1.7285e-05 - val_accuracy: 1.0000


In [83]:
results = {}
for root, dirs, files in os.walk("test_images"):
    for filename in files:
        sss = training_set.class_indices
#         cv.imread('images/brands/filename')
#         print(filename)
        test_image = image.load_img(f"test_images/{filename}", target_size=(96, 96))

        test_image = image.img_to_array(test_image)
#         test_image = test_image / 255

        test_image = np.expand_dims(test_image, axis=0)

        result = np.argmax(classifier.predict(test_image))
        for k in sss.keys():
            if sss[k] == result:
                bn = k

        results[filename] = bn
        
#         print(result)

In [84]:
type(training_set.class_indices)

dict

In [85]:
results

{'2382afd2208b4f08a8758cbfa84b7ab2.jpg': 'brands',
 '014baffbb4064ac9871a8c18643dedaf.jpg': 'brands',
 '96e9dd37bccd4a9e908f31d331468e04.jpg': 'brands',
 'a6541d71d57247e4a78ad72e31d24607.jpg': 'brands',
 '022a8fba5d4e4c0d8ce5fc10eadd61b8.jpg': 'brands',
 '228ddef65d3147339669d3ff9a93f09a.jpg': 'brands',
 'c13eafe7303444f093df8225a6034af5.jpg': 'brands',
 'b31e3a985737413b9e0241476c247f57.jpg': 'brands',
 'ad285d96b2b44ef9951d29340c7e193d.jpg': 'brands',
 '296f89362d3c422cb18b6282426ae55b.jpg': 'brands',
 '835956ec598b486ea8aff0b8675712d4.jpg': 'brands',
 '4ffb919b7bfd498ea485754d937e91b5.jpg': 'brands',
 'd54a352a2cce49c3b0f13b805c17d00d.jpg': 'brands',
 'ed1d9c1e91b44525b4441bf92a88251c.jpg': 'brands',
 '6575fdf9c2a648be92041b73e98200cf.jpg': 'brands',
 '3920185a4a05484c8f31337ddcde4dad.jpg': 'brands',
 'e4f9e001b0ee47f7b7c2dea0b8d61081.jpg': 'brands',
 'f35372a4098242f1a5d027e848f9636c.jpg': 'brands',
 '4620f286c7aa4ab9b9f1cd8ee93b1a19.jpg': 'brands',
 'f37f37e1271d473894d4aa7f83805

In [59]:
ke = list(results.keys())
bnn = list(results.values())

In [60]:
print(ke[0])
print(bnn[0])

2382afd2208b4f08a8758cbfa84b7ab2.jpg
brands


In [63]:
df = pd.DataFrame()

In [64]:
a_file = open("res.csv", "w")
# a_dict = {"a": 1, "b": 2}

writer = csv.writer(a_file)
for key, value in results.items():
    writer.writerow([key, value])

a_file.close()

In [65]:
training_set